<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemple d'apprentissage du ET avec TensorFlow


##Importer les librairires

In [0]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

---

***Définir les entrées et sorties attendues***


In [0]:
entrees = np.array([[0,0],[0,1],[1,0],[1,1]], float)
sorties = np.array([[0],[0],[0],[1]], float)

---

***Choisir le modèle de réseau*** (*ici les couches sont séquentielles*)

In [0]:
model = Sequential()

---

***Définir l'architecture du réseau*** (*ici ajout d'une couche constituée de 1 neurone en sortie, de 3 neurones en entrée (2 contenant les valeurs + un Bias (émettant toujours le signal 1)), utilisation de la sigmoïde comme fonction d'activation*)

In [0]:
avec_bias = True
model.add(Dense(1, input_dim=2, use_bias=avec_bias, activation='sigmoid'))

---

***Compiler le  réseau*** * (ici, on précise que l'algo de correction d'erreur est 'Adamax', et que l'erreur calculée est la moyenne des valeurs absolues des erreurs commises)*

In [0]:
model.compile(optimizer='adamax', loss='MAE')

---

***Entraîner le réseau*** (*ici on ne le fait pas parler, et on lance 10000 cylces d'apprentissage*)

In [14]:
model.fit(entrees, sorties, verbose=0, epochs=10000)

---

***Vérifier le réseau*** (*Etape facultative, en générale on le teste sur d'autres exemples. Ici, on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement*)

In [0]:
predictions = model.predict(entrees)

---
**Affichage des résultats** (*on affiche les entrées, la sortie attendue, la sortie calculées ainsi que les poids appliquées aux entrées et au signal bias*) 

In [16]:
print("verification")
for i in range(0, len(entrees)):
    print(entrees[i][0], " - ", entrees[i][1], " attendu ", sorties[i], " trouvé ",  predictions[i])

ws = model.get_weights()
print("poids pour entree x = " + str(ws[0][0][0]))
print("poids pour entree y = " + str(ws[0][1][0]))
if(avec_bias):print("poids pour bias = " + str(ws[1][0]))

verification
0.0  -  0.0  attendu  [0.]  trouvé  [1.8517728e-09]
0.0  -  1.0  attendu  [0.]  trouvé  [0.00122396]
1.0  -  0.0  attendu  [0.]  trouvé  [0.00122068]
1.0  -  1.0  attendu  [1.]  trouvé  [0.9987651]
poids pour entree x = 13.399996
poids pour entree y = 13.402682
poids pour bias = -20.107122
